In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Source

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Market Basket Analysis/Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
  
df['InvoiceNo'] = df['InvoiceNo'].astype('str') 
df = df[~df['InvoiceNo'].str.contains('C')]

In [ ]:
basket = (df[df['Country'] =="France"].groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))
basket.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,15CM CHRISTMAS GLASS BALL 20 LIGHTS,16 PIECE CUTLERY SET PANTRY DESIGN,18PC WOODEN CUTLERY SET DISPOSABLE,20 DOLLY PEGS RETROSPOT,200 RED + WHITE BENDY STRAWS,3 HOOK HANGER MAGIC GARDEN,3 PIECE SPACEBOY COOKIE CUTTER SET,3 RAFFIA RIBBONS 50'S CHRISTMAS,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN RED AND CREAM,3 TRADITIONAl BISCUIT CUTTERS SET,36 DOILIES DOLLY GIRL,36 DOILIES VINTAGE CHRISTMAS,36 FOIL HEART CAKE CASES,36 FOIL STAR CAKE CASES,36 PENCILS TUBE RED RETROSPOT,36 PENCILS TUBE SKULLS,36 PENCILS TUBE WOODLAND,3D DOG PICTURE PLAYING CARDS,3D HEARTS HONEYCOMB PAPER GARLAND,3D SHEET OF DOG STICKERS,3D TRADITIONAL CHRISTMAS STICKERS,3D VINTAGE CHRISTMAS STICKERS,4 IVORY DINNER CANDLES SILVER FLOCK,4 PINK DINNER CANDLE SILVER FLOCK,4 TRADITIONAL SPINNING TOPS,5 HOOK HANGER MAGIC TOADSTOOL,5 HOOK HANGER RED MAGIC TOADSTOOL,50'S CHRISTMAS GIFT BAG LARGE,6 GIFT TAGS 50'S CHRISTMAS,...,WOODLAND DESIGN COTTON TOTE BAG,WOODLAND LARGE BLUE FELT HEART,WOODLAND LARGE PINK FELT HEART,WOODLAND LARGE RED FELT HEART,WOODLAND MINI BACKPACK,WOODLAND PARTY BAG + STICKER SET,WOODLAND SMALL BLUE FELT HEART,WOODLAND SMALL PINK FELT HEART,WOODLAND SMALL RED FELT HEART,WOODLAND STORAGE BOX LARGE,WOODLAND STORAGE BOX SMALL,WORLD WAR 2 GLIDERS ASSTD DESIGNS,WRAP VINTAGE DOILY,WRAP 50'S CHRISTMAS,WRAP ALPHABET DESIGN,WRAP CAROUSEL,WRAP CHRISTMAS VILLAGE,WRAP CIRCUS PARADE,WRAP DOILEY DESIGN,WRAP DOLLY GIRL,WRAP ENGLISH ROSE,WRAP GINGHAM ROSE,WRAP GREEN PEARS,WRAP I LOVE LONDON,WRAP PAISLEY PARK,WRAP PINK FAIRY CAKES,WRAP POPPIES DESIGN,WRAP RED APPLES,WRAP RED VINTAGE DOILY,WRAP SUKI AND FRIENDS,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
basket_sets = basket.applymap(encode_units)
 
#menghapus kolom postage
basket_sets.drop('POSTAGE', inplace=True, axis=1)
basket_sets.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,15CM CHRISTMAS GLASS BALL 20 LIGHTS,16 PIECE CUTLERY SET PANTRY DESIGN,18PC WOODEN CUTLERY SET DISPOSABLE,20 DOLLY PEGS RETROSPOT,200 RED + WHITE BENDY STRAWS,3 HOOK HANGER MAGIC GARDEN,3 PIECE SPACEBOY COOKIE CUTTER SET,3 RAFFIA RIBBONS 50'S CHRISTMAS,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN RED AND CREAM,3 TRADITIONAl BISCUIT CUTTERS SET,36 DOILIES DOLLY GIRL,36 DOILIES VINTAGE CHRISTMAS,36 FOIL HEART CAKE CASES,36 FOIL STAR CAKE CASES,36 PENCILS TUBE RED RETROSPOT,36 PENCILS TUBE SKULLS,36 PENCILS TUBE WOODLAND,3D DOG PICTURE PLAYING CARDS,3D HEARTS HONEYCOMB PAPER GARLAND,3D SHEET OF DOG STICKERS,3D TRADITIONAL CHRISTMAS STICKERS,3D VINTAGE CHRISTMAS STICKERS,4 IVORY DINNER CANDLES SILVER FLOCK,4 PINK DINNER CANDLE SILVER FLOCK,4 TRADITIONAL SPINNING TOPS,5 HOOK HANGER MAGIC TOADSTOOL,5 HOOK HANGER RED MAGIC TOADSTOOL,50'S CHRISTMAS GIFT BAG LARGE,6 GIFT TAGS 50'S CHRISTMAS,...,WOODLAND DESIGN COTTON TOTE BAG,WOODLAND LARGE BLUE FELT HEART,WOODLAND LARGE PINK FELT HEART,WOODLAND LARGE RED FELT HEART,WOODLAND MINI BACKPACK,WOODLAND PARTY BAG + STICKER SET,WOODLAND SMALL BLUE FELT HEART,WOODLAND SMALL PINK FELT HEART,WOODLAND SMALL RED FELT HEART,WOODLAND STORAGE BOX LARGE,WOODLAND STORAGE BOX SMALL,WORLD WAR 2 GLIDERS ASSTD DESIGNS,WRAP VINTAGE DOILY,WRAP 50'S CHRISTMAS,WRAP ALPHABET DESIGN,WRAP CAROUSEL,WRAP CHRISTMAS VILLAGE,WRAP CIRCUS PARADE,WRAP DOILEY DESIGN,WRAP DOLLY GIRL,WRAP ENGLISH ROSE,WRAP GINGHAM ROSE,WRAP GREEN PEARS,WRAP I LOVE LONDON,WRAP PAISLEY PARK,WRAP PINK FAIRY CAKES,WRAP POPPIES DESIGN,WRAP RED APPLES,WRAP RED VINTAGE DOILY,WRAP SUKI AND FRIENDS,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Build up the frequent items 
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.071429,(4 TRADITIONAL SPINNING TOPS)
1,0.096939,(ALARM CLOCK BAKELIKE GREEN)
2,0.102041,(ALARM CLOCK BAKELIKE PINK)
3,0.094388,(ALARM CLOCK BAKELIKE RED)
4,0.081633,(BAKING SET 9 PIECE RETROSPOT)


In [ ]:
# Create the rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.094388,0.102041,0.073980,0.783784,7.681081,0.064348,4.153061
5,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.102041,0.094388,0.073980,0.725000,7.681081,0.064348,3.293135
6,(DOLLY GIRL LUNCH BOX),(SPACEBOY LUNCH BOX),0.099490,0.125000,0.071429,0.717949,5.743590,0.058992,3.102273
7,(SPACEBOY LUNCH BOX),(DOLLY GIRL LUNCH BOX),0.125000,0.099490,0.071429,0.571429,5.743590,0.058992,2.101190
8,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN SPACEBOY),0.168367,0.137755,0.089286,0.530303,3.849607,0.066092,1.835747
9,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN CIRCUS PARADE),0.137755,0.168367,0.089286,0.648148,3.849607,0.066092,2.363588


In [ ]:
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
16,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS),0.127551,0.132653,0.102041,0.800000,6.030769,0.085121,4.336735
18,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.137755,0.122449,0.960000,6.968889,0.104878,21.556122
19,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.137755,0.127551,0.122449,0.888889,6.968889,0.104878,7.852041
20,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975000,7.077778,0.085433,34.489796
21,"(SET/6 RED SPOTTY PAPER PLATES, SET/6 RED SPOT...",(SET/20 RED RETROSPOT PAPER NAPKINS),0.122449,0.132653,0.099490,0.812500,6.125000,0.083247,4.625850
22,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975000,7.644000,0.086474,34.897959


In [ ]:
df_final = rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

In [ ]:
df_final.to_csv('/content/drive/MyDrive/Market Basket Analysis/outputMBA.csv')

# Data STI

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Market Basket Analysis/prod/STI/STI_Market_Basket_Analysis_11.csv')
df.head()

,CustomerID,ItemID,ItemName,Store,QTY,PostingDate,Description
0,100002.0,102535.0,FOX,PDG,1.0,28-11-20 0:00,FOX LEM 168
1,100002.0,102578.0,MC,PDG,3.0,06-12-19,MC ACRYLIC 3MM L000
2,100002.0,102908.0,NIPPON,PDG,20.0,07-11-19,NIPPON RESIBON D.WHEEL 4IN BSH
3,100002.0,102908.0,NIPPON,PDG,19.0,18-05-19,NIPPON RESIBON D.WHEEL 4IN BSH
4,100002.0,102908.0,NIPPON,PDG,15.0,03-09-19,NIPPON RESIBON D.WHEEL 4IN BSH


In [ ]:
df['ItemID'] = df['ItemID'].astype(int)
#df['Description'] = df['Description'].str.strip()
#df['ItemName'] = df['ItemName'].str.strip()
df.dropna(axis=0, subset=['CustomerID'], inplace=True)
  
df['CustomerID'] = df['CustomerID'].astype('str') 
#df = df[~df['InvoiceID'].str.contains('C')]

In [ ]:
#basket = (df[df['Store'] =="PDG"].groupby(['CustomerID', 'Description'])['QTY'].sum().unstack().reset_index().fillna(0).set_index('CustomerID'))
basket = (df[df['Store'] =="PDG"].groupby(['CustomerID', 'ItemID'])['QTY'].sum().unstack().reset_index().fillna(0).set_index('CustomerID'))
basket.head()

ItemID,100003,100008,100011,100012,100044,100082,100092,100158,100159,100160,100169,100173,100197,100198,100200,100201,100202,100203,100204,100205,100206,100209,100218,100219,100220,100222,100228,100231,100237,100238,100239,100243,100245,100247,100250,100251,100253,100255,100269,100270,...,121366,121367,121368,121370,121371,121372,121373,121375,121376,121377,121378,121379,121380,121384,121387,121407,121408,121409,121418,121423,121424,121425,121426,121427,121428,121429,121431,121432,121433,121435,121436,121437,121438,121444,121445,121447,121449,121470,121471,121472
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100004.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
100008.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100012.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 0.1:
        return 1
basket_sets = basket.applymap(encode_units)
 
#menghapus kolom postage
#basket_sets.drop('POSTAGE', inplace=True, axis=1)
basket_sets.head()

ItemID,100003,100008,100011,100012,100044,100082,100092,100158,100159,100160,100169,100173,100197,100198,100200,100201,100202,100203,100204,100205,100206,100209,100218,100219,100220,100222,100228,100231,100237,100238,100239,100243,100245,100247,100250,100251,100253,100255,100269,100270,...,121366,121367,121368,121370,121371,121372,121373,121375,121376,121377,121378,121379,121380,121384,121387,121407,121408,121409,121418,121423,121424,121425,121426,121427,121428,121429,121431,121432,121433,121435,121436,121437,121438,121444,121445,121447,121449,121470,121471,121472
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100004.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
100008.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100011.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100012.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
basket_sets.isnull().sum().sum()
#basket.isnull().values.any()

0

In [ ]:
# Build up the frequent items 
frequent_itemsets = apriori(basket_sets, min_support=0.06, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.060367,(100473)
1,0.125984,(100706)
2,0.135171,(100836)
3,0.110236,(100951)
4,0.209974,(102535)


In [ ]:
# Create the rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.8)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(100706),(100836),0.125984,0.135171,0.062992,0.500000,3.699029,0.045963,1.729659
1,(100836),(100706),0.135171,0.125984,0.062992,0.466019,3.699029,0.045963,1.636793
2,(103161),(102535),0.229659,0.209974,0.102362,0.445714,2.122714,0.054140,1.425305
3,(102535),(103161),0.209974,0.229659,0.102362,0.487500,2.122714,0.054140,1.503105
4,(103162),(102535),0.220472,0.209974,0.078740,0.357143,1.700893,0.032447,1.228930
...,...,...,...,...,...,...,...,...,...
255,"(109225, 109223)",(109221),0.081365,0.099738,0.060367,0.741935,7.438879,0.052252,3.488517
256,"(109221, 109223)",(109225),0.072178,0.133858,0.060367,0.836364,6.248128,0.050706,5.293088
257,(109225),"(109221, 109223)",0.133858,0.072178,0.060367,0.450980,6.248128,0.050706,1.689961
258,(109221),"(109225, 109223)",0.099738,0.081365,0.060367,0.605263,7.438879,0.052252,2.327209


In [ ]:
rules[ (rules['lift'] >= 1) &
       (rules['confidence'] >= 0.4) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(100706),(100836),0.125984,0.135171,0.062992,0.500000,3.699029,0.045963,1.729659
1,(100836),(100706),0.135171,0.125984,0.062992,0.466019,3.699029,0.045963,1.636793
2,(103161),(102535),0.229659,0.209974,0.102362,0.445714,2.122714,0.054140,1.425305
3,(102535),(103161),0.209974,0.229659,0.102362,0.487500,2.122714,0.054140,1.503105
6,(104134),(102535),0.167979,0.209974,0.097113,0.578125,2.753320,0.061842,1.872655
...,...,...,...,...,...,...,...,...,...
255,"(109225, 109223)",(109221),0.081365,0.099738,0.060367,0.741935,7.438879,0.052252,3.488517
256,"(109221, 109223)",(109225),0.072178,0.133858,0.060367,0.836364,6.248128,0.050706,5.293088
257,(109225),"(109221, 109223)",0.133858,0.072178,0.060367,0.450980,6.248128,0.050706,1.689961
258,(109221),"(109225, 109223)",0.099738,0.081365,0.060367,0.605263,7.438879,0.052252,2.327209


In [ ]:
output = rules[ (rules['lift'] >= 1) &
       (rules['confidence'] >= 0.4) ]

In [ ]:
output.to_csv('/content/drive/MyDrive/Market Basket Analysis/prod/outputMBA_STI_ItemID.csv', index = False)

# Data KHI

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Market Basket Analysis/prod/KHI/KHI_Market_Basket_Analysis_11.csv')
df.head()

,InvoiceID,ItemID,ItemName,Store,QTY,PostingDate,Description
0,F180000305,100164,KLASSEN,JKT,300,01-08-18,KLASSEN SL.3351 REL LACI 35CM
1,F180000305,100165,KLASSEN,JKT,300,01-08-18,KLASSEN SL.3351 REL LACI 40CM
2,F180000305,100166,KLASSEN,JKT,300,01-08-18,KLASSEN SL.3351 REL LACI 45CM
3,F180000306,100217,HPL,JKT,20,01-08-18,HPL 001 AA
4,F180000306,100218,HPL,JKT,20,01-08-18,HPL 002 AA


In [ ]:
df['ItemID'] = df['ItemID'].astype(int) 
#df['Description'] = df['Description'].str.strip()
#df['ItemName'] = df['ItemName'].str.strip()
df.dropna(axis=0, subset=['InvoiceID'], inplace=True)
  
#df['CustomerID'] = df['CustomerID'].astype('str') 
df['InvoiceID'] = df['InvoiceID'].astype('str') 
#df = df[~df['InvoiceID'].str.contains('C')]

In [ ]:
basket = (df[df['Store'] =="JKT"].groupby(['InvoiceID', 'ItemID'])['QTY'].sum().unstack().reset_index().fillna(0).set_index('InvoiceID'))
basket.head()

ItemID,100001,100002,100003,100005,100006,100007,100008,100009,100010,100014,100016,100017,100018,100020,100021,100022,100023,100024,100025,100026,100027,100028,100029,100030,100031,100032,100033,100034,100035,100036,100037,100039,100040,100041,100042,100044,100045,100046,100047,100048,...,100193,100194,100195,100196,100197,100198,100199,100200,100201,100202,100203,100204,100205,100206,100207,100208,100209,100210,100211,100212,100213,100214,100215,100216,100217,100218,100219,100220,100221,100222,100223,100224,100225,100226,100227,100228,100229,100230,100231,100232
InvoiceID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
F180000305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F180000306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F180000307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F180000308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F180000309,0.0,0.0,0.0,200.0,200.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 0.1:
        return 1
basket_sets = basket.applymap(encode_units)
 
#menghapus kolom postage
#basket_sets.drop('POSTAGE', inplace=True, axis=1)
basket_sets.head()

ItemID,100001,100002,100003,100005,100006,100007,100008,100009,100010,100014,100016,100017,100018,100020,100021,100022,100023,100024,100025,100026,100027,100028,100029,100030,100031,100032,100033,100034,100035,100036,100037,100039,100040,100041,100042,100044,100045,100046,100047,100048,...,100193,100194,100195,100196,100197,100198,100199,100200,100201,100202,100203,100204,100205,100206,100207,100208,100209,100210,100211,100212,100213,100214,100215,100216,100217,100218,100219,100220,100221,100222,100223,100224,100225,100226,100227,100228,100229,100230,100231,100232
InvoiceID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
F180000305,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
F180000306,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
F180000307,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
F180000308,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
F180000309,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
basket_sets.isnull().sum().sum()
#basket.isnull().values.any()

0

In [ ]:
# Build up the frequent items 
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.314319,(100037)
1,0.183353,(100071)
2,0.188591,(100097)
3,0.195576,(100130)
4,0.210710,(100131)


In [ ]:
# Create the rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(100037),(100071),0.314319,0.183353,0.158906,0.505556,2.757284,0.101274,1.651646
1,(100071),(100037),0.183353,0.314319,0.158906,0.866667,2.757284,0.101274,5.142608
2,(100097),(100037),0.188591,0.314319,0.166473,0.882716,2.808345,0.107195,5.846333
3,(100037),(100097),0.314319,0.188591,0.166473,0.529630,2.808345,0.107195,1.725042
4,(100037),(100165),0.314319,0.196740,0.086147,0.274074,1.393075,0.024307,1.106531
...,...,...,...,...,...,...,...,...,...
99,"(100165, 100167)",(100166),0.104773,0.190338,0.087893,0.838889,4.407373,0.067951,5.025491
100,"(100166, 100167)",(100165),0.112922,0.196740,0.087893,0.778351,3.956231,0.065677,3.624008
101,(100165),"(100166, 100167)",0.196740,0.112922,0.087893,0.446746,3.956231,0.065677,1.603382
102,(100166),"(100165, 100167)",0.190338,0.104773,0.087893,0.461774,4.407373,0.067951,1.663291


In [ ]:
rules[ (rules['lift'] >=1) &
       (rules['confidence'] >= 0.2) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(100037),(100071),0.314319,0.183353,0.158906,0.505556,2.757284,0.101274,1.651646
1,(100071),(100037),0.183353,0.314319,0.158906,0.866667,2.757284,0.101274,5.142608
2,(100097),(100037),0.188591,0.314319,0.166473,0.882716,2.808345,0.107195,5.846333
3,(100037),(100097),0.314319,0.188591,0.166473,0.529630,2.808345,0.107195,1.725042
4,(100037),(100165),0.314319,0.196740,0.086147,0.274074,1.393075,0.024307,1.106531
...,...,...,...,...,...,...,...,...,...
99,"(100165, 100167)",(100166),0.104773,0.190338,0.087893,0.838889,4.407373,0.067951,5.025491
100,"(100166, 100167)",(100165),0.112922,0.196740,0.087893,0.778351,3.956231,0.065677,3.624008
101,(100165),"(100166, 100167)",0.196740,0.112922,0.087893,0.446746,3.956231,0.065677,1.603382
102,(100166),"(100165, 100167)",0.190338,0.104773,0.087893,0.461774,4.407373,0.067951,1.663291


In [ ]:
output = rules[ (rules['lift'] >= 1) &
       (rules['confidence'] >= 0.2) ]

In [ ]:
output.to_csv('/content/drive/MyDrive/Market Basket Analysis/prod/outputMBA_KHI_ItemID.csv', index = False)

# USD

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Market Basket Analysis/prod/USD/USD_Market_Basket_Analysis_11.csv')
df.head()

,CustomerID,ItemID,ItemName,Store,QTY,PostingDate,Description
0,100002,100006,IKAD,PLM,8,29-05-20,IKAD DX 3133
1,100002,100015,IKAD,PLM,2,20-01-20,IKAD GE 7415
2,100002,100018,IKAD,PLM,10,16-11-19,IKAD GE 7621
3,100002,100083,IKAD,PLM,6,22-01-20,IKAD SX 8575
4,100002,100083,IKAD,PLM,3,24-01-20,IKAD SX 8575


In [ ]:
df['ItemID'] = df['ItemID'].astype(int)
#df['Description'] = df['Description'].str.strip()
#df['ItemName'] = df['ItemName'].str.strip()
df.dropna(axis=0, subset=['CustomerID'], inplace=True)
  
df['CustomerID'] = df['CustomerID'].astype('str') 
#df['InvoiceID'] = df['InvoiceID'].astype('str') 
#df = df[~df['InvoiceID'].str.contains('C')]

In [ ]:
#basket = (df[df['Store'] =="PDG"].groupby(['CustomerID', 'Description'])['QTY'].sum().unstack().reset_index().fillna(0).set_index('CustomerID'))
basket = (df[df['Store'] =="PDG"].groupby(['CustomerID', 'ItemID'])['QTY'].sum().unstack().reset_index().fillna(0).set_index('CustomerID'))
basket.head()

ItemID,100006,100015,100016,100017,100020,100022,100032,100033,100041,100044,100046,100049,100056,100057,100066,100076,100077,100078,100083,100085,100086,100088,100089,100090,100092,100093,100094,100095,100097,100098,100100,100101,100103,100112,100117,100122,100123,100129,100174,100175,...,101166,101167,101168,101169,101170,101171,101172,101173,101175,101176,101177,101178,101179,101180,101181,101182,101183,101184,101185,101186,101187,101190,101191,101192,101193,101194,101195,101197,101198,101199,101200,101202,101203,101204,101205,101206,101208,101209,101210,101211
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100477,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100478,2.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,101.0,0.0,0.0,66.0,0.0,0.0,6.0,0.0,0.0,18.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100479,603.0,160.0,62.0,0.0,0.0,0.0,0.0,0.0,356.0,3.0,0.0,0.0,230.0,0.0,6.0,0.0,318.0,41.0,415.0,6.0,71.0,137.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,50.0,41.0,0.0,0.0,0.0,...,248.0,0.0,83.0,0.0,0.0,0.0,0.0,0.0,0.0,556.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0,0.0,0.0,0.0,0.0,0.0,74.0,135.0,0.0,4.0,0.0,42.0,40.0,2.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0
100480,12.0,64.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,10.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 0.1:
        return 1
basket_sets = basket.applymap(encode_units)
 
#menghapus kolom postage
#basket_sets.drop('POSTAGE', inplace=True, axis=1)
basket_sets.head()

ItemID,100006,100015,100016,100017,100020,100022,100032,100033,100041,100044,100046,100049,100056,100057,100066,100076,100077,100078,100083,100085,100086,100088,100089,100090,100092,100093,100094,100095,100097,100098,100100,100101,100103,100112,100117,100122,100123,100129,100174,100175,...,101166,101167,101168,101169,101170,101171,101172,101173,101175,101176,101177,101178,101179,101180,101181,101182,101183,101184,101185,101186,101187,101190,101191,101192,101193,101194,101195,101197,101198,101199,101200,101202,101203,101204,101205,101206,101208,101209,101210,101211
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100477,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100478,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
100479,1,1,1,0,0,0,0,0,1,1,0,0,1,0,1,0,1,1,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,...,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,0,1,1,1,1,0,0,0,0,0,0,1
100480,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100481,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
basket_sets.isnull().sum().sum()
#basket.isnull().values.any()

0

In [ ]:
# Build up the frequent items 
frequent_itemsets = apriori(basket_sets, min_support=0.3, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.300000,(100041)
1,0.652381,(100056)
2,0.323810,(100066)
3,0.342857,(100323)
4,0.580952,(100627)


In [ ]:
# Create the rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(100056),(100627),0.652381,0.580952,0.428571,0.656934,1.130789,0.049569,1.221479
1,(100627),(100056),0.580952,0.652381,0.428571,0.737705,1.130789,0.049569,1.325298
2,(100056),(100629),0.652381,0.580952,0.452381,0.693431,1.193610,0.073379,1.366893
3,(100629),(100056),0.580952,0.652381,0.452381,0.778689,1.193610,0.073379,1.570723
4,(100640),(100056),0.547619,0.652381,0.423810,0.773913,1.186290,0.066553,1.537546
...,...,...,...,...,...,...,...,...,...
1025,"(100629, 100871)","(100832, 101066)",0.419048,0.385714,0.300000,0.715909,1.856061,0.138367,2.162286
1026,(100832),"(101066, 100629, 100871)",0.433333,0.385714,0.300000,0.692308,1.794872,0.132857,1.996429
1027,(101066),"(100832, 100629, 100871)",0.647619,0.323810,0.300000,0.463235,1.430580,0.090295,1.259752
1028,(100629),"(100832, 101066, 100871)",0.580952,0.319048,0.300000,0.516393,1.618547,0.114649,1.408071


In [ ]:
rules[ (rules['lift'] >=1) &
       (rules['confidence'] >= 0.8) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
9,(100825),(100056),0.490476,0.652381,0.400000,0.815534,1.250089,0.080023,1.884461
11,(100832),(100056),0.433333,0.652381,0.371429,0.857143,1.313869,0.088730,2.433333
15,(100851),(100056),0.409524,0.652381,0.328571,0.802326,1.229842,0.061406,1.758543
19,(100904),(100056),0.371429,0.652381,0.309524,0.833333,1.277372,0.067211,2.085714
21,(100907),(100056),0.361905,0.652381,0.300000,0.828947,1.270649,0.063900,2.032234
...,...,...,...,...,...,...,...,...,...
1006,"(100832, 100825)","(101066, 100629)",0.361905,0.485714,0.300000,0.828947,1.706656,0.124218,3.006593
1016,"(100832, 101066, 100629)",(100871),0.357143,0.571429,0.300000,0.840000,1.470000,0.095918,2.678571
1017,"(100832, 101066, 100871)",(100629),0.319048,0.580952,0.300000,0.940299,1.618547,0.114649,7.019048
1018,"(100832, 100629, 100871)",(101066),0.323810,0.647619,0.300000,0.926471,1.430580,0.090295,4.792381


In [ ]:
output = rules[ (rules['lift'] >= 1) &
       (rules['confidence'] >= 0.8) ]

In [ ]:
#output.to_csv('/content/drive/MyDrive/Market Basket Analysis/outputMBA_USD.csv', index = False)
output.to_csv('/content/drive/MyDrive/Market Basket Analysis/prod/outputMBA_USD_ItemID.csv', index = False)